In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append(2*'../')

import os
import glob
from pathlib import Path
from omegaconf import DictConfig
import yaml

import torch
import lightning as L

from rl4co.tasks.rl4co import RL4COLitModule
from rl4co.utils.lightning import load_model_from_checkpoint
from rl4co.tasks.eval import evaluate_policy


/home/botu/botu/Dev/rl4co/env/lib/python3.10/site-packages/torchrl/__init__.py:26: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


## Loading model from checkpoint

In [2]:
checkpoints_path = Path('../../saved_checkpoints/')

exp_name = 'tsp50'
model_name = 'am-tsp50'
checkpoints_path = checkpoints_path / exp_name / model_name

cfg_path = checkpoints_path / 'config.yaml'
ckpt_path = checkpoints_path / 'epoch_099.ckpt'

lit_module = load_model_from_checkpoint(cfg_path, ckpt_path, phase='test')

Unused kwargs: {'params': {'total': 708608, 'trainable': 708608, 'non_trainable': 0}}


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

policy = lit_module.model.policy.to(device)
policy.eval()
env = lit_module.model.env

In [10]:
test_dataset = lit_module.test_dataset

# Greedy
out_greedy = evaluate_policy(env, policy, test_dataset, method="greedy")

# Sampling
out_samppling = evaluate_policy(env, policy, test_dataset, method="sampling", samples=50)

# Greedy multistart
out_greedy_ms = evaluate_policy(env, policy, test_dataset, method="greedy_multistart")#, num_starts=20)

# Augment dihedral 8 (same as POMO)
out_augd = evaluate_policy(env, policy, test_dataset, method="augment_dihedral_8")

# Symmetric augment
out_aug = evaluate_policy(env, policy, test_dataset, method="augment", num_augment=50)

# Greedy multistart with augment dihedral 8
out_greedy_ms_augd = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment_dihedral_8")

# Greedy multistart with symmetric augment
out_greddy_ms_aug = evaluate_policy(env, policy, test_dataset, method="greedy_multistart_augment", num_augment=50)

Effective batch size: 4096 (ratio: 1)
Using automatic batch size: 4096


Running greedy:   0%|          | 0/3 [00:00<?, ?it/s]

Mean reward for greedy: -5.7785
Time: 0.5225s
Effective batch size: 512 (ratio: 50)
Using automatic batch size: 512


Running sampling:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for sampling: -5.7339
Time: 18.9920s
Effective batch size: 4096 (ratio: 5)
Using automatic batch size: 4096


...

Mean reward for augmentation: -5.7050
Time: 19.1802s
Effective batch size: 512 (ratio: 40)
Using automatic batch size: 512


Running greedy_multistart_augment:   0%|          | 0/20 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7146
Time: 14.7996s
Effective batch size: 128 (ratio: 250)
Using automatic batch size: 128


Running greedy_multistart_augment:   0%|          | 0/79 [00:00<?, ?it/s]

Mean reward for greedy_multistart_augment: -5.7022
Time: 91.2396s
